# Convertir etiquetado personalizado a formato YOLO Darknet

> Input
*   Directorio que contiene las imagenes y el archivo de etiquetado en formato JSON
*   Ancho de las imagenes del dataset 
*   Alto de las imagenes del dataset 

> Output
* Carpeta 'labels' con etiquetas del dataset en formato YOLO Darknet (un archivo TXT con etiquetas para cada imagen)


In [1]:
#parametros iniciales necesarios

directorio = '/content/drive/MyDrive/PT/dataset/IMG_1119'    #directorio que contiene las imagenes y el archivo de etiquetado en formato json
w_i = 1920    #ancho de imagenes en entero, ej: 3264
h_i = 1080    #alto de imagenes en entero, ej: 2448

carpeta = 'labels_comp'    #carpeta donde se almacenara el nuevo etiquetado generado     

In [2]:
#importacion de bibliotecas necesarias

import json
import os
import errno
from PIL import Image
from PIL.ExifTags import TAGS   

In [3]:
#definicion de variables 

w = w_i
h = h_i    #se asignan valores recibidos de ancho y alto a variables auxiliares 
directorio = directorio.replace('\\', '/') + "/"    #se modifica el directorio para correcta lectura de python
contenido_directorio = os.listdir(directorio)
data = {}
nombre_archivos_jpg = []        
ejpg = '.jpg'
ejson = '.json'
etxt = '.txt'
#archivo_json = 'berries.json'    #nombre del archivo que contiene el etiquetado original

In [4]:
#comprobacion de archivos en directorio

try:
    os.mkdir(directorio + carpeta)    #se crea una carpeta para almacenar todos los archivos de etiquetado creados
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

for archivo in contenido_directorio:
    if os.path.isfile(os.path.join(directorio, archivo)) and archivo.endswith(ejpg):
        nombre_archivos_jpg.append(archivo.replace(ejpg, ''))    #se obtienen los nombres de los archivos JPG encontrados y se almacenan en una lista sin la extension '.jpg'
        print("Archivo JPG encontrado: ", archivo)
        
for archivo in contenido_directorio:
    if os.path.isfile(os.path.join(directorio, archivo)) and archivo.endswith(ejson):
      archivo_json = archivo    #se obtiene el nombre del primer archivo JSON encontrado y se almacena en la variable archivo_json
      print("Archivo JSON encontrado: ", archivo)
      break

etiquetas = directorio + archivo_json

Archivo JPG encontrado:  frame0000.jpg
Archivo JPG encontrado:  frame0150.jpg
Archivo JPG encontrado:  frame0250.jpg
Archivo JPG encontrado:  frame0318.jpg
Archivo JPG encontrado:  frame0375.jpg
Archivo JPG encontrado:  frame0425.jpg
Archivo JPG encontrado:  frame0495.jpg
Archivo JPG encontrado:  frame0540.jpg
Archivo JPG encontrado:  frame0585.jpg
Archivo JPG encontrado:  frame0650.jpg
Archivo JPG encontrado:  frame0750.jpg
Archivo JPG encontrado:  frame0782.jpg
Archivo JPG encontrado:  frame0800.jpg
Archivo JPG encontrado:  frame0930.jpg
Archivo JPG encontrado:  frame0969.jpg
Archivo JPG encontrado:  frame0988.jpg
Archivo JPG encontrado:  frame1000.jpg
Archivo JPG encontrado:  frame1050.jpg
Archivo JPG encontrado:  frame1085.jpg
Archivo JPG encontrado:  frame1150.jpg
Archivo JPG encontrado:  frame1145.jpg
Archivo JPG encontrado:  frame1130.jpg
Archivo JPG encontrado:  frame1158.jpg
Archivo JPG encontrado:  frame1177.jpg
Archivo JPG encontrado:  frame1217.jpg
Archivo JPG encontrado:  

In [5]:
#generacion de etiquetas
cuenta = 0

with open(etiquetas) as file:
    data = json.load(file)

    for nombre in nombre_archivos_jpg:    #para cada nombre almacenado en 'nombre_archivos_jpg', obtenidos a partir del dataset de imagenes
        puntos = []
        lineas = []
        archivo_txt = directorio + carpeta + '/' + nombre + etxt    #se genera un archivo '.txt' para almacenar el etiquetado en formato 'YOLO DARKNET TXT'
        imagen = nombre + ejpg    #imagen es el "objeto" a buscar en el json (nombre de la imagen, cuyo etiquetado esta contenido en el objeto)
        archivo_jpg = directorio + imagen    #ubicacion de cada imagen para lectura de metadatos
        image = Image.open(archivo_jpg)    #Lee la imagen en el directorio señalado
        exifdata = image.getexif() 

        for tagid in exifdata: 
            tagname = TAGS.get(tagid, tagid) 
            value = exifdata.get(tagid) 
            if (tagname == "Orientation" and value == 1):    #si la orientacion de la imagen es horizontal
                w = w_i
                h = h_i
            if (tagname == "Orientation" and value == 6):    #si la orientacion de la imagen es vertical
                w = h_i
                h = w_i
            #print(f"{tagname:25}: {value}") 
        image.close()

        try:    #se usa try-except en caso de que no exista etiquetado en el json para alguna imagen del dataset
            for objeto in data[imagen]:
                cuenta = cuenta + 1
                puntos.append("0")    #se agrega a la lista un 0 (clase a la que pertenece el etiquetado), y luego todos los puntos del bounding box

                punto_x = int(objeto['x'])    #los puntos obtenidos del archivo json se transforman a formato numerico
                punto_y = int(objeto['y'])
                punto_w = int(objeto['w'])
                punto_h = int(objeto['h'])

                bbox_w = (punto_w - punto_x)/w    #coordenadas normalizadas segun requerimientos de YOLO 
                bbox_h = (punto_h - punto_y)/h
                bbox_x = (punto_x/w + bbox_w/2)
                bbox_y = (punto_y/h + bbox_h/2)

                puntos.append(str("{0:.6f}".format(bbox_x)))    #se limita la cantidad de decimales en las coordenadas normalizadas 
                puntos.append(str("{0:.6f}".format(bbox_y)))    #y se almacenan en una lista como string
                puntos.append(str("{0:.6f}".format(bbox_w))) 
                puntos.append(str("{0:.6f}".format(bbox_h))) 
                
                lineas.append(" ".join(puntos))    #la lista de puntos previa es un bounding box y este se almacenan en otra lista
                puntos = []

        except KeyError:    #se controla excepcion en caso de que no exista etiquetado en json para alguna imagen del dataset
            lineas = ""    #se eliminan los puntos almacenados del archivo anterior
            archivo_txt = ""    #se elimina el nombre del archivo sin etiquetado
            print("Error en: ", imagen)
        print("cuenta = ", cuenta)
        if archivo_txt != "":    #no se consideran los nombres de archivo sin etiquetado
            print("======================================")
            print("Imagen: ", archivo_jpg)
            print("Lineas generadas (etiquetas): ")
            with open(archivo_txt, "w") as file:
                for linea in lineas:
                    print(linea)
                    file.write(linea + "\n")    #se escriben los puntos de los bounding box en el archivo '.txt'
            print("Archivo de texto generado: ", archivo_txt)
            print("======================================")

            WINDOWS_LINE_ENDING = b'\r\n'
            UNIX_LINE_ENDING = b'\n'
            with open(archivo_txt, 'rb') as open_file:
                content = open_file.read()
                content = content.replace(WINDOWS_LINE_ENDING, UNIX_LINE_ENDING)    #conversion de formato de endline windows->unix
            with open(archivo_txt, 'wb') as open_file:
                open_file.write(content)


cuenta =  47
Imagen:  /content/drive/MyDrive/PT/dataset/IMG_1119/frame0000.jpg
Lineas generadas (etiquetas): 
0 0.178646 0.233333 0.055208 0.096296
0 0.206771 0.287963 0.036458 0.064815
0 0.176562 0.319444 0.048958 0.083333
0 0.247396 0.288889 0.048958 0.085185
0 0.255208 0.226852 0.047917 0.075926
0 0.236979 0.379630 0.046875 0.081481
0 0.269792 0.370370 0.031250 0.048148
0 0.256250 0.456481 0.047917 0.087037
0 0.315104 0.396296 0.038542 0.070370
0 0.299479 0.483333 0.040625 0.059259
0 0.335417 0.441667 0.033333 0.061111
0 0.361979 0.458333 0.036458 0.068519
0 0.358854 0.335185 0.057292 0.100000
0 0.371875 0.255556 0.039583 0.066667
0 0.403125 0.314815 0.031250 0.074074
0 0.407292 0.412037 0.045833 0.083333
0 0.392188 0.494444 0.038542 0.074074
0 0.453125 0.348148 0.041667 0.062963
0 0.423438 0.540741 0.044792 0.085185
0 0.456771 0.469444 0.040625 0.068519
0 0.472917 0.276852 0.050000 0.087037
0 0.522396 0.286111 0.048958 0.083333
0 0.488542 0.416667 0.054167 0.092593
0 0.573438 0.413